# PREPROCESAMIENTO DE DATOS
1. Lectura y Formateo
2. Selección
3. Preparación

In [1]:
## Get current work directory to load all the custom modules
from pathlib import Path
import sys
BASE_DIR = Path().absolute().resolve(strict=True).parent
if str(BASE_DIR) not in sys.path:
    sys.path.append(str(BASE_DIR))

In [2]:
DATA_PATH = os.path.join(BASE_DIR, 'data', 'autos')
print(DATA_PATH)

/workspace/Externos/TimeSeries/BenchMark/data/autos


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
import plotly.express as px
from sklearn.impute import SimpleImputer
from matplotlib.pyplot import figure
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
import re 

In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import linear_model
from sklearn import svm
from sklearn.model_selection import cross_val_score
from statsmodels.tsa.stattools import adfuller

In [21]:
from helpers.general import homogenizar_str, print_linea_de_tiempo_producto
from datetime import datetime, timedelta
import datetime as dt

># 1. Lectura y Formateo de datos

In [6]:
# Leemos los datos
df = pd.read_csv(f"{DATA_PATH}/Data_Repuestos_2018_2021.csv", header=1, 
                       usecols=[' noPeriodo',' idArticulo','CnSalida 2018', ' caDescProduct'],
                       )
df.head()

,noPeriodo,idArticulo,CnSalida 2018,caDescProduct
0,20180100,25319,1.0,FOCO H11-12 100-90W CURVO
1,20180100,4923,1.0,FILTRO AIRE HYUNDAI TUCSON
2,20180100,25853,1.0,FILTRO GASOLINA NISSAN METAL GA15-16 INJECTADO
3,20180100,13497,1.0,JGO CABLES BUJIA KIA RIO LARGO
4,20180100,23900,4.0,BUJIA NGK


In [7]:
# Cambiamos el nombre de las columnas
df.rename(columns={
    ' noPeriodo':'Periodo',
    ' idArticulo':'producto',
    'CnSalida 2018':'ventas',
    ' caDescProduct':'DescProducto'
    }, inplace=True)
df.head()

,Periodo,producto,ventas,DescProducto
0,20180100,25319,1.0,FOCO H11-12 100-90W CURVO
1,20180100,4923,1.0,FILTRO AIRE HYUNDAI TUCSON
2,20180100,25853,1.0,FILTRO GASOLINA NISSAN METAL GA15-16 INJECTADO
3,20180100,13497,1.0,JGO CABLES BUJIA KIA RIO LARGO
4,20180100,23900,4.0,BUJIA NGK


### Formating datetime

In [8]:
# Formaterar las fechas
df['producto'] = df['producto'].astype(str)
df.Periodo = df.Periodo.apply(lambda x: datetime.strptime(str(x), "%Y%m00").date())
df.head()

,Periodo,producto,ventas,DescProducto
0,2018-01-01,25319,1.0,FOCO H11-12 100-90W CURVO
1,2018-01-01,4923,1.0,FILTRO AIRE HYUNDAI TUCSON
2,2018-01-01,25853,1.0,FILTRO GASOLINA NISSAN METAL GA15-16 INJECTADO
3,2018-01-01,13497,1.0,JGO CABLES BUJIA KIA RIO LARGO
4,2018-01-01,23900,4.0,BUJIA NGK


In [9]:
# # Homogenizamos los nombres de los productos
# df["DescProductoOrig"] = df.loc[:,"DescProducto"]
# df["DescProducto"] = df["DescProducto"].apply(homogenizar_str)
# df.head()

In [10]:
# Ver el estado de salud de los datos
### Buscamos valores nulos 
### observamos el tipo de datos en la db
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99954 entries, 0 to 99953
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Periodo       99954 non-null  object 
 1   producto      99954 non-null  object 
 2   ventas        99954 non-null  float64
 3   DescProducto  99951 non-null  object 
dtypes: float64(1), object(3)
memory usage: 3.1+ MB


In [11]:
# Solo valores significativos
df = df.query('ventas >= 0')
df.describe()

,ventas
count,99954.000000
mean,5.407716
std,87.734119
min,0.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,10000.000000


## Group by products and months
The group process going to sum all the sales in the related month and product.

In [13]:
# Agrupas datos según mes, año y producto
dfagg = df.groupby(['Periodo', 'producto'])
dfagg = dfagg.agg({'ventas': 'sum', 'DescProducto':'first'})
dfagg = dfagg.reset_index()
dfagg = dfagg.sort_values(by=['producto', 'Periodo'], ascending=False)
dfagg.head()

,Periodo,producto,ventas,DescProducto
60387,2021-03-01,9996,1.0,THERMOSTATO TOYOTA COROLLA 1C/2C/2Y/3Y/4Y/3A/5...
54016,2020-11-01,9996,1.0,THERMOSTATO TOYOTA COROLLA 1C/2C/2Y/3Y/4Y/3A/5...
47091,2020-07-01,9996,4.0,THERMOSTATO TOYOTA COROLLA 1C/2C/2Y/3Y/4Y/3A/5...
42002,2020-02-01,9996,1.0,THERMOSTATO TOYOTA COROLLA 1C/2C/2Y/3Y/4Y/3A/5...
35136,2019-10-01,9996,1.0,THERMOSTATO TOYOTA COROLLA 1C/2C/2Y/3Y/4Y/3A/5...


In [25]:
# seleccionando productos que se encuentran al menos en el año 2021
## de esta manera se podrán realizar predicciones en el futuro.
products21 =dfagg[dfagg['Periodo']>=dt.date(year=2021,month=1,day=1)]['producto'].unique().tolist()

># 2. Seleccion de datos

## Productos y su presencia en el tiempo

- Cada año tiene 36 meses.
- Tomaremos aquellos productos con al menos 12 meses de presencia en la línea de tiempo, es decir en el últimos 3 años.
- detectamos que tenemos 49 artículos únicos que cumplen todos los requisitos

In [35]:
min_meses = 36

In [36]:
# Aplicamos el filtro para encontrar los productos significativos
temp = dfagg.loc[dfagg['producto'].isin(products21)]
temp = temp.loc[:,['producto']]
temp['count'] = 1
temp = temp.groupby('producto').count()
temp = temp.sort_values('count', ascending=False)
temp = temp.reset_index()
temp = temp.query(f'count > {min_meses}')
value_count = temp.producto.nunique()

print(f"Existen {value_count} producto que cuentan con una presencia mayor a {min_meses} meses")

Existen 49 producto que cuentan con una presencia mayor a 36 meses


In [39]:
# Tomamos solo los productos que cumples con las condiciones deseadas.
lista_articulos = temp.producto.unique().tolist()
df_clean = dfagg.loc[dfagg['producto'].isin(lista_articulos)]
df_clean = df_clean.reset_index(drop=True)
df_clean = df_clean.sort_values(by=['producto', 'Periodo'], ascending=False)
df_clean.rename(columns={'producto': 'idArticulo'}, inplace=True)
df_clean.head()

,Periodo,idArticulo,ventas,DescProducto
0,2021-05-01,9950,37.0,REFRIGERANTE COOLANT AZUL/NATURAL TRANSP. GALO...
1,2021-04-01,9950,19.0,REFRIGERANTE COOLANT AZUL/NATURAL TRANSP. GALO...
2,2021-03-01,9950,9.0,REFRIGERANTE COOLANT AZUL/NATURAL TRANSP. GALO...
3,2021-02-01,9950,8.0,REFRIGERANTE COOLANT AZUL/NATURAL TRANSP. GALO...
4,2021-01-01,9950,76.0,REFRIGERANTE COOLANT AZUL/NATURAL TRANSP. GALO...


In [40]:
df_clean.to_csv(f'{DATA_PATH}/producto.csv', index=False)

># 3. Preparación

### Crear series de tiempo

In [41]:
# Pivotando las tablas y llenando de ceros las ventas que no existen.
imp_mean = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
df_time_pre = df_clean.pivot_table(index='Periodo', columns='idArticulo', values='ventas', aggfunc='sum',)
df_time = imp_mean.fit_transform(df_time_pre)
df_time = pd.DataFrame(df_time, columns=df_time_pre.columns, index=df_time_pre.index)
df_time.tail()

idArticulo,10702,10927,10960,10964,11395,11397,11509,11523,11807,1208,...,5389,551,5579,594,6833,7798,7799,7801,9776,9950
Periodo,,,,,,,,,,,,,,,,,,,,,
2021-01-01,7.0,326.0,19.0,9.0,271.0,44.0,2.0,2.0,4.0,170.0,...,5.0,6.0,6.0,3.0,1.0,6.0,9.0,5.0,1840.0,76.0
2021-02-01,4.0,12.0,13.0,6.0,40.0,0.0,2.0,4.0,18.0,220.0,...,0.0,4.0,3.0,4.0,1.0,2.0,1.0,5.0,840.0,8.0
2021-03-01,2.0,29.0,13.0,16.0,94.0,4.0,3.0,4.0,6.0,155.0,...,2.0,4.0,4.0,6.0,3.0,2.0,5.0,3.0,480.0,9.0
2021-04-01,8.0,39.0,101.0,11.0,51.0,8.0,1.0,2.0,4.0,155.0,...,1.0,8.0,5.0,8.0,1.0,5.0,4.0,4.0,450.0,19.0
2021-05-01,5.0,4.0,15.0,10.0,64.0,12.0,2.0,2.0,1.0,80.0,...,1.0,13.0,11.0,4.0,1.0,5.0,3.0,0.0,260.0,37.0


### Graficando los productos en la línea de tiempo

In [43]:
scaler = StandardScaler().fit(df_time)
df_time_scaled = scaler.transform(df_time)
df_time_scaled = pd.DataFrame(df_time_scaled, columns=df_time.columns, index=df_time.index)
df_time_scaled.head()
print_linea_de_tiempo_producto(df_time_scaled , 'idArticulo', height=2000)